### Модели LLM с использованием внешних данных

## Установка зависимостей для GGML

In [ ]:
!pip install langchain==0.0.275 pydantic==1.10.13 openai==0.28.1 faiss-cpu==1.7.4 tiktoken==0.5.1 sentence_transformers==2.2.2 nltk==3.8.1
!pip install -U deep-translator==1.11.4

In [ ]:
import os
os.environ["LANGCHAIN_TRACING"] = "true" # If you want to trace the execution of the program, set to "true"
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python==0.1.77 --no-cache-dir

## Установка зависимостей для GGUF




In [ ]:
!pip install langchain==0.0.322 pydantic openai==0.28.1 faiss-cpu==1.7.4 tiktoken==0.5.1 sentence_transformers==2.2.2 nltk==3.8.1
!pip install -U deep-translator==1.11.4

In [ ]:
import os
os.environ["LANGCHAIN_TRACING"] = "true" # If you want to trace the execution of the program, set to "true"
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python==0.2.11 --no-cache-dir

In [ ]:
import os
import pandas as pd
import pickle
import tqdm
import re
import gdown
import tiktoken
from deep_translator import GoogleTranslator
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, NLTKTextSplitter
# import nltk
# nltk.download('punkt')

from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain
from google.colab import userdata

TOTAL_AMOUNT = 0
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
def load_googledoc_by_url(url, path):
    id = url.split('/')[-2]
    gdown.download(f'https://drive.google.com/uc?export=download&id={id}', path)


# Подсчет токенов
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))


# количество токенов индексной базы и стоимость. model Ada v2
def tokens_count_db_index(source_chunks):
    global TOTAL_AMOUNT
    count_tokens = 0
    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    count_tokens += count_token
    print(f'Количество токенов в документе: {count_token}')
    # Embedding model Ada v2 - $0.00010 / 1K tokens - 28/02/2024 - https://openai.com/pricing
    price = 0.0001 * count_tokens / 1000
    TOTAL_AMOUNT += price
    print(f'ЦЕНА создания индексной базы: $ {price}')

In [ ]:
# База знаний:
url = 'https://drive.google.com/file/d/1-gRMM-kX0WbCwFWzSr-6mvj-HZx9tGFf/view?usp=sharing '
load_googledoc_by_url(url, 'knowledge_base.txt')

loader = TextLoader("/content/knowledge_base.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
tokens_count_db_index(docs)

# Чанки на русском
with open('docs_ru.pkl', "wb") as file:
    pickle.dump(docs, file)
len(docs)

Downloading...
From: https://drive.google.com/uc?export=download&id=1-gRMM-kX0WbCwFWzSr-6mvj-HZx9tGFf
To: /content/knowledge_base.txt
100%|██████████| 1.24M/1.24M [00:00<00:00, 129MB/s]


Количество токенов в документе: 274618
ЦЕНА создания индексной базы: $ 0.0274618


1015

### на русском

In [ ]:
# Количество токенов в документе: 274618
# более 150 тыс. - не пройдет лимит по бесплатному токену OpenAI
# Разделим текстовую базу чанков на две и отдельно создадим индексные базы
import time

db1 = FAISS.from_documents(docs[:450], OpenAIEmbeddings())
time.sleep(60)
db2 = FAISS.from_documents(docs[450:], OpenAIEmbeddings())
time.sleep(60)
db_ru = FAISS.from_documents(db1.docstore._dict.values(), OpenAIEmbeddings())
db_ru.merge_from(db2)

db_ru.save_local(folder_path='', index_name='db_ru')
len(db_ru.docstore._dict)

1015

### на английском

In [ ]:
with open('docs_ru.pkl', "rb") as file:
    docs = pickle.load(file) # Текстовые чанки на русском

docs_translate = []
for doc in tqdm.tqdm(docs):
  docs_translate.append(Document(page_content=GoogleTranslator(source='ru', target='en').translate(doc.page_content)))

# Текстовые чанки на английском
with open('docs_en.pkl', "wb") as file:
    pickle.dump(docs_translate, file)

In [ ]:
tokens_count_db_index(docs_translate)

Количество токенов в документе: 134153
ЦЕНА создания индексной базы: $ 0.0134153


In [ ]:
db_en = FAISS.from_documents(docs_translate, OpenAIEmbeddings())
db_en.save_local(folder_path='', index_name='db_en')
len(db_en.docstore._dict)

1015

## Загружаем вопросы

In [ ]:
question_url = 'https://drive.google.com/file/d/1DyVljnB5K5cHX_Mod0P9CPxBE6syGH4n/view'
load_googledoc_by_url(question_url, 'Вопросы.csv')

df_qw = pd.read_csv("/content/Вопросы.csv", delimiter='/n')
df_qw['Вопрос'].values

Downloading...
From: https://drive.google.com/uc?export=download&id=1DyVljnB5K5cHX_Mod0P9CPxBE6syGH4n
To: /content/Вопросы.csv
100%|██████████| 1.47k/1.47k [00:00<00:00, 1.49MB/s]
<ipython-input-5-5546a5639496>:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_qw = pd.read_csv("/content/Вопросы.csv", delimiter='/n')


array(['Как настраивать chatGPT?',
       'Какие варианты рассрочки есть на тариф Основной?',
       'Какие курсы по chatGPT есть в УИИ?',
       '"Какие темы включает в себя курс ""AI Директор""?"',
       'Какой документ будет по окончанию образования?',
       'Курс AI директор сколько стоит?',
       '"Можно ли купить стажировки, если я не являюсь студентом УИИ?"',
       'Можно проходить обучение с телефона?',
       'На каком языке проходит обучение?',
       'Несовершеннолетний ребенок может проходить обучение?',
       'Нужно ли мне покупать видеокарту для занятий?',
       'Почему выгодно внедрять chatGPT?', 'Расскажите про курс AutoML',
       'С кем сотрудничает УИИ и куда трудоустраивают выпускников после прохождения обучения?',
       'Сколько стоит использование ChatGPT?',
       'Что включает в себя тариф AI под ключ?',
       'Что значит гарантия трудоустройства?',
       'Что такое Business AI Club?', 'Что такое промты?',
       'Я живу в другом часовом поясе. Смогу ли

## Llama-2-7B-chat GGML 4 - RU

In [ ]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_1.bin

--2024-02-29 17:31:56--  https://huggingface.co/TheBloke/Llama-2-7B-chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_1.bin
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.88, 18.172.134.124, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_1.bin [following]
--2024-02-29 17:31:56--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_1.bin
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/b379a454d32593089f663914c5a4687d95fd37e3bcddf826f9fb107598a8ff50?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q4_1.bin%3B+filename%3D%22llama-2-7b-chat.ggml

In [ ]:
n_gpu_layers = 20000  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llama = LlamaCpp(
    model_path="/content/llama-2-7b-chat.ggmlv3.q4_1.bin",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=4096,  # Context window
    max_tokens=1000,  # Max tokens to generate
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
template = """<s>[INST] <<SYS>>Ты менеджер поддержки в чате Российской компании Университет Исскуственного интеллекта. Компания продает курсы по AI.
У компании есть большой документ со всеми материалами о продуктах компании на русском языке. Тебе задает вопрос клиент в чате, дай ему ответ на языке оригинала,
опираясь на отрывки из этого документа, постарайся ответить так, чтобы человек захотел после ответа купить обучение. Отвечай максимально
точно по документу, не придумывай ничего от себя. Никогда не ссылайся на название документа или названия его отрывков при ответе, клиент ничего не должен
знать о документе, по которому ты отвечаешь. Отвечай от первого лица без ссылок на источники на которые ты опираешься. Если ты не знаешь ответа или его нет в документе, то ответь "Я не могу отетить на этот вопрос".
<</SYS>>

Вопрос клиента:
{question}

Документ с информацией для ответа клиенту:
{docs}

[/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["question", 'docs'])

db = FAISS.load_local(folder_path='',
                      embeddings=OpenAIEmbeddings(),
                      index_name='db_ru')

In [ ]:
%%time
llm_chain = LLMChain(prompt=prompt, llm=llama)
history = []

for query in df_qw['Вопрос'].values[15:]: # 6 последних вопросов
    sim_docs = db.similarity_search(query)
    docs = re.sub(r'\n{2}', ' ', '\n '.join([f'======\n' + doc.page_content + '\n' for i, doc in enumerate(sim_docs)]))
    result = llm_chain.run({'question':query, 'docs': docs})
    history.append([str(query), GoogleTranslator(source='auto', target='ru').translate(result), str(docs)])

pd.DataFrame(history, columns=['Вопрос', 'Ответ', 'Контекст']).\
to_csv(os.path.join('', 'Llama-2-7B-Chat-GGML-4' + '.csv'), index=False)

Thank you for providing the information! As a customer support manager in the University of Intelligent Machines, I would recommend the "AI under the Key" tariff to the client. This tariff offers a comprehensive and inclusive learning experience, with a variety of benefits that can help the client achieve their goals.
Firstly, the "AI under the Key" tariff includes 32 sessions on neural networks, which is a fundamental aspect of AI. These sessions are designed to provide a deep understanding of the subject matter, and the client will have the opportunity to practice and apply their knowledge throughout the course.
Secondly, the tariff offers support from a curator for a period of 3 years and 8 months, providing the client with ongoing guidance and mentorship. This is particularly valuable, as AI can be a complex and rapidly evolving field, and having a knowledgeable guide can make all the difference in mastering the subject.
Thirdly, the "AI under the Key" tariff includes a guarantee o

Llama.generate: prefix-match hit


Добрый день! Я рад помочь вам в ответе на вопрос客та. Гарантия трудоустройства - это обещание компании Университета Искусственного Интеллекта (УИИ) обеспечить работу своим выпускникам после прохождения обучения на определенных направлениях.
В документе указаны следующие меры, которые компания принимает для поддержки молодых специалистов в IT-сфере:
1. освобождение от уплаты налога на прибыль в течение 3 лет;
2. освобождение от уплаты 13% налога на доходы физических лиц в течение 3 лет;
3. предоставление кредитов по льготной ставке 3%.
Кроме того, вы можете использовать возможность получения отсрочки от призыва, чтобы сосредоточиться на развитии своей карьеры в IT-индустрии. В целом, компания УИИ предлагает множество возможностей для молодых людей, которые хотят получить знания и навыки в области AI.
Я не могу оценить качество документа, так как это запрещено моим инструментами. Однако, я могу сказать, что документ содержит информацию, которая поможет вам лучше понять, какие навыки и зна

Llama.generate: prefix-match hit


Добрый день! Спасибо за вопрос. Как менеджер поддержки, я могу обязательно помочь вам с ответом на ваш вопрос.
Вот мой ответ:
Business AI Club - это проект Университета Интеллектуального Интегрального Образования (УИИ), который был создан для применения Artificial Intelligence в бизнесе. Проект предлагает компаниям возможность использовать AI для решения различных проблем, таких как automatization, predictive analytics, natural language processing и другие.
Международный хакатон "AI for Good" - это соревнование, в котором команды разрабатывают прототипы IT-продуктов для решения определенных проблем, связанных с использованием AI для добра и благополучия людей. Участие в хакатонах позволяет продвинуть свои проекты, прокачать навыки и завести полезные знакомства, а также получить часть призового фонда.
Участники Business AI Club могут принимать участие в хакатонах, чтобы продемонстрировать свои проекты и получить признание на рынке IT. Это может помочь им заключить договоры с крупными ко

Llama.generate: prefix-match hit


Добрый день! Я рад помочь вам в ответе на этот вопрос.
Вот как я могу ответить на этот вопрос, основанный на информации из документа:
Промты (prompts) - это инструкции на естественном языке, предназначенные для обучения и управления ChatGPT. Они помогают разработчикам и пользователям взаимодействовать с моделью, передавая ей конкретные задачи и вопросы, чтобы она могла ответить на них более точно и эффективно.
Промты-инжиринг (prompt engineering) - это процесс разработки промтов для обучения ChatGPT. Это похоже на программирование, но требует более высокого уровня абстракции и не является четким.
Существует несколько ролей в работе с ChatGPT: разработчик, копирайтер, промт-инженер, тестировщик и архитектор базы знаний.
Курс по обучению ChatGPT включает в себя 2 основных занятия:
1. Основы программирования на Python - 3 занятия
2. Аккаунт ChatGPT - 1 занятие

Стоимость обращений к ChatGPT невысокая, и при текущем курсе доллара средняя стоимость одного запроса составляет 50 копеек.
В nea

Llama.generate: prefix-match hit


Для вас, как клиента, я могу ответить следующим образом:
Ваш интерес к обучению в Университете Исскучного Интеллекта (УИИ) не является выдумкой. В документе, который вы предоставили, содержатся реальные истории успешных студентов, которые также начали учиться в УИИ с нуля и достигли значительных успехов.
Ваше желание работы удаленно, с нормальной зарплатой и независимо от начальства, также является очень реальным. Многочисленные студенты и преподаватели УИИ работают удаленно и достигают значительных успехов в своей карьере.
Существует множество преимуществ, которые вы получаете при обучении в УИИ, включая возможность самостоятельно решать, в какое время работать, а также возможность работы на половинной работной ставке. Кроме того, ученые и эксперты УИИ готовы помочь вам в любое время, не обходясь с вами.
Не забудьте, что вы можете обратиться за помощью к поддержке любойtime, не отражаясь на том, чтобы вы получили доступ к обучению и поддержке.
В целом, я считаю, что ваш выбор на обуче

NameError: name 'project_path' is not defined

In [ ]:
# Поправил ошибку с 'project_path'

pd.DataFrame(history, columns=['Вопрос', 'Ответ', 'Контекст']).\
to_csv(os.path.join('', 'Llama-2-7B-Chat-GGML-4' + '.csv'), index=False)

## StableBeluga 2 13B GGUF - RU

In [ ]:
!wget https://huggingface.co/TheBloke/StableBeluga-13B-GGUF/resolve/main/stablebeluga-13b.Q4_0.gguf

--2024-02-29 18:26:38--  https://huggingface.co/TheBloke/StableBeluga-13B-GGUF/resolve/main/stablebeluga-13b.Q4_0.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.124, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/93/22/9322f2586fc694f5a2c313ee51ce942b1a9d0efbd1c15c42a3575ac0fcaa7c1b/54a8982cfa6e062d2ebf8ec4d79eb782b9c42fe20b0a698d8520b56f1c48bcae?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27stablebeluga-13b.Q4_0.gguf%3B+filename%3D%22stablebeluga-13b.Q4_0.gguf%22%3B&Expires=1709490398&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTQ5MDM5OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85My8yMi85MzIyZjI1ODZmYzY5NGY1YTJjMzEzZWU1MWNlOTQyYjFhOWQwZWZiZDFjMTVjNDJhMzU3NWFjMGZjYWE3YzFiLzU0YTg5ODJjZmE2ZTA2MmQyZWJmOGVjNGQ3OWViNzgyYjljN

In [ ]:
n_gpu_layers = 1000  # Metal set to 1 is enough.
n_batch = 256  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llama = LlamaCpp(
    model_path="/content/stablebeluga-13b.Q4_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    temperature=0,
    n_ctx=4096,  # Context window
    max_tokens=1000,  # Max tokens to generate
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
template = """### System: \nТы менеджер поддержки в чате Российской компании Университет Исскуственного интеллекта. Компания продает курсы по AI.
У компании есть большой документ со всеми материалами о продуктах компании на русском языке. Тебе задает вопрос клиент в чате, дай ему ответ на языке оригинала,
опираясь на отрывки из этого документа, постарайся ответить так, чтобы человек захотел после ответа купить обучение. Отвечай максимально
точно по документу, не придумывай ничего от себя. Никогда не ссылайся на название документа или названия его отрывков при ответе, клиент ничего не должен
знать о документе, по которому ты отвечаешь. Отвечай от первого лица без ссылок на источники на которые ты опираешься. Если ты не знаешь ответа или его нет в документе, то ответь "Я не могу отетить на этот вопрос".\n\n
### User: \nВопрос клиента:
{question}\n
Документ с информацией для ответа клиенту:
{docs}\n\n
### Assistant:\n
"""

prompt = PromptTemplate(template=template, input_variables=["question", 'docs'])

db = FAISS.load_local(folder_path='',
                      embeddings=OpenAIEmbeddings(),
                      index_name='db_ru')

In [ ]:
%%time
llm_chain = LLMChain(prompt=prompt, llm=llama)
history = []

for query in df_qw['Вопрос'].values[15:]: # 6 последних вопросов
  sim_docs = db.similarity_search(query)
  docs = re.sub(r'\n{2}', ' ', '\n '.join([f'======\n' + doc.page_content + '\n' for i, doc in enumerate(sim_docs)]))
  result = llm_chain.run({'question':query, 'docs': docs})
  history.append([str(query), result, str(docs)])

pd.DataFrame(history, columns=['Вопрос', 'Ответ', 'Контекст']).\
to_csv(os.path.join('', 'StableBeluga-2-13B-GGUF-4' + '.csv'), index=False)

Тариф AI под ключ включает в себя следующие возможности:

1. 32 занятия по нейронным сетям;
2. Поддержку куратора на протяжении 3 лет и 8 месяцев;
3. Гарантию трудоустройства;
4. Реализацию AI проекта;
5. 36 продвинутых занятий по нейронным сетям;
6. 25 бонусных занятий;
7. Курс "Python для анализа данных";
8. Курс "AI директор";
9. 60 часов индивидуальных консультаций;
10. Курс "Python разработчик";
11. 12 месяцев стажировок.

Llama.generate: prefix-match hit


Вопрос клиента:
Что значит гарантия трудоустройства?

Ответ: Гарантия трудоустройства - это официальное обязательство компании Университета Исскуственного интеллекта (УИИ) обеспечить своим выпускникам возможность трудоустроиться в соответствующей сфере. Это означает, что после успешного прохождения обучения и получения сертификата, компания гарантирует вам трудоустройство в соответствующей области.

Документ с информацией для ответа клиенту:

Университет Исскуственного интеллекта (УИИ) имеет собственное HR-агентство Al hunter и трудоустраивает выпускников в российские и зарубежные компании. Официальная гарантия трудоустройства прописана в договоре. После успешного прохождения студентом от 32 занятий, сдачи экзамена и получения сертификата, УИИ обязуется:

1. Обеспечить вам возможность трудоустроиться в соответствующей области;
2. Помочь вам найти работу в соответствующей сфере;
3. Провести личные собеседования с компаниями, которые могут предложить вам работу.

Таким образом, УИИ гаран

Llama.generate: prefix-match hit


Business AI Club - это проект, который будет влиять на всю нишу AI в России. Этот клуб собирает специалистов, занимающих высокие должности в разных компаниях, которые принимают решение о внедрении AI. В клубе уже 1034 регистрации.

В рамках проекта проводятся живые встречи участников клуба с трансляцией в Zoom раз в месяц. На этих встречах обсуждаются идеи применения AI, кейсы применения, расходы на проекты и оценки проектов.

Цель - применение AI компаниями, получение AI заказов от компаний.

В рамках проекта также проводятся хакатоны, в которых студенты УИИ регулярно принимают участие и занимают призовые места.

Llama.generate: prefix-match hit


Что такое промты? Промты - это инструкции на естественном языке, предназначенные для ChatGPT. Этот процесс называется промт-инжиринг и похож на программирование, но требует более высокого уровня абстракции и не является четким.

Как будет происходить тестирование ChatGPT? Будет создан список из 20-100 вопросов, на которые модель должна дать ответы. Помимо этого, сама модель может задавать вопросы для тестирования. Финальная проверка производится людьми, которые оценивают правильность ответов. Если модель ошибается, то изменяются промты или исходный датасет. Такой подход позволяет постоянно улучшать качество работы ChatGPT.

Чем этот курс полезен для студентов? Дообучение ChatGPT открывает широкий спектр возможностей для применения этой технологии в различных областях. Благодаря дообучению ChatGPT, пользователи могут расширить функциональность речевой модели и улучшить качество ответов в соответствии с конкретными потребностями.

Где можно применять дообучение ChatGPT? Этот курс поможет

Llama.generate: prefix-match hit


Да, вы можете учиться в Университете Исскуственного интеллекта, даже если вы живете в другом часовом поясе. Курсы предоставляются онлайн и вы можете пройти обучение по своему графику.


CPU times: user 1min 28s, sys: 15.7 s, total: 1min 43s
Wall time: 1min 48s


## StableBeluga 2 13B GGML - EN (с переводом)

In [ ]:
!wget https://huggingface.co/TheBloke/StableBeluga-13B-GGML/resolve/main/stablebeluga-13b.ggmlv3.q4_1.bin

--2024-02-29 18:13:12--  https://huggingface.co/TheBloke/StableBeluga-13B-GGML/resolve/main/stablebeluga-13b.ggmlv3.q4_1.bin
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.124, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/78/a9/78a9edd70b7383e401f5fad995be8125c0360b874fcc97b30a9baf51366db065/8b42e0e125058bcc0a0a18d707793809aa0cd3a74e691e8f35dae415b09b5298?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27stablebeluga-13b.ggmlv3.q4_1.bin%3B+filename%3D%22stablebeluga-13b.ggmlv3.q4_1.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1709489593&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTQ4OTU5M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83OC9hOS83OGE5ZWRkNzBiNzM4M2U0MDFmNWZhZDk5NWJlODEyNWMwMzYwYjg3NGZjYzk3YjMwYTli

In [ ]:
n_gpu_layers = 4000  # Metal set to 1 is enough.
n_batch = 256  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llama = LlamaCpp(
    model_path="/content/stablebeluga-13b.ggmlv3.q4_1.bin",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    temperature=0,
    n_ctx=4096,  # Context window
    max_tokens=1000,  # Max tokens to generate
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
template = """### System: \nYou are the chat support manager of the Russian company University of Artificial Intelligence. The company sells courses on AI.
The company has a large document with all the materials about the company's products in Russian. A client asks you a question in a chat, give him an answer based on excerpts from this document, try to answer in such a way that the person wants to buy training after answering. Answer as much as possible
exactly according to the document, do not invent anything on your own. Never refer to the title of a document or the title of its passages when answering; the client does not owe anything
know about the document for which you are responding. Answer in the first person without citing the sources you rely on. If you do not know the answer or it is not in the document, then answer “I cannot answer this question.”\n\n
### User: \nClient question:
{question}\n
Document with information to respond to the client:
{docs}\n\n
### Assistant:\n
"""

prompt = PromptTemplate(template=template, input_variables=["question", 'docs'])

db = FAISS.load_local(folder_path='',
                      embeddings=OpenAIEmbeddings(),
                      index_name='db_en')

In [ ]:
%%time
llm_chain = LLMChain(prompt=prompt, llm=llama)
history = []

for query in df_qw['Вопрос'].values[15:]: # 6 последних вопросов
    sim_docs = db.similarity_search(GoogleTranslator(source='auto', target='en').translate(query))
    docs = re.sub(r'\n{2}', ' ', '\n '.join([f'"======"\n' + doc.page_content + '\n' for i, doc in enumerate(sim_docs)]))
    result = llm_chain.run({'question':GoogleTranslator(source='auto', target='en').translate(query), 'docs': docs})
    history.append([str(query), GoogleTranslator(source='auto', target='ru').translate(result),
                    GoogleTranslator(source='auto', target='ru').translate(docs)])

pd.DataFrame(history, columns=['Вопрос', 'Ответ', 'Контекст']).\
to_csv(os.path.join('', 'StableBeluga-2-13B-GGML-4-en' + '.csv'), index=False)

The AI ​​turnkey plan includes various aspects related to developing an AI project and gaining experience in the field of artificial intelligence. Some key features include:

1. Introductory part: how the AI project works, datasets, video cards.
2. Main part: Development of an AI project in 10 days in reality mode (analysis of the experiment log, list of next hypotheses, detailed analysis of topics and technologies used in the project).
3. Employment in AI
4. Collection of a neural network for classification of satellite images in Terra AI
5. Webinar on AI reality
6. UII students will receive information on various topics related to AI projects, such as choosing a development path, setting goals, and selecting a project topic.
7. Participation conditions for company projects: free project development (neural network and integration), 3-month duration, intellectual rights transferred to the company, base and markup from the company (possibly from UII but without guarantee), and UII's ri

Llama.generate: prefix-match hit


Job security means providing a stable and reliable source of income for individuals in their careers. In the context of University of Artificial Intelligence, job security is achieved through various initiatives such as creating a stable team specializing in answering questions, handling calls, assessing quality, organizing webinars, and mailing. This allows students to find work on outsourced projects and eventually transition into self-employment or employment with partner companies. The IT sector is a source of stability, reliability, and constant demand, providing numerous opportunities for young people aged 18 to 27 years old. Additionally, affordable mortgage programs with an interest rate of only 5% provide financial stability in the future for young IT professionals. AI specialists are respected by society and business because they are engaged in the most progressive type of activity.

Llama.generate: prefix-match hit


Business AI Club is a project that will influence the entire AI niche in Russia. It gathers specialists holding high positions in different companies who decide to implement AI. The goal of the club is to create an AI project and hire developers from among UII graduates. The business side of AI will be covered, including application cases, costs, project evaluations, implementation ideas, developer interviews, and more. Live meetings of club members are held once a month in Moscow with Zoom broadcasts.

Llama.generate: prefix-match hit


Industrial products are goods that are produced in large quantities, often using automated processes, and are designed for use in various industries or sectors such as manufacturing, construction, transportation, and energy. These products can include machinery, equipment, tools, components, and materials used in the production of other goods or services. Industrial products play a crucial role in driving economic growth and productivity by enabling businesses to produce more efficiently and effectively.

Llama.generate: prefix-match hit


I cannot answer this questionCPU times: user 59.7 s, sys: 22.8 s, total: 1min 22s
Wall time: 1min 39s


## Объединение результатов

In [ ]:
import os
import pandas as pd

project_path = "/content/csv/"
files = os.listdir(project_path)
files

['StableBeluga-2-13B-GGML-4-en.csv',
 'Llama-2-7B-Chat-GGML-4.csv',
 'StableBeluga-2-13B-GGUF-4.csv']

In [ ]:
# Создаем пустой DataFrame с нужной структурой
df_all = pd.DataFrame(columns=['Вопрос', 'Модель', 'Ответ', 'Контекст'])

for file in files:
    df = pd.read_csv(os.path.join(project_path, file))
    # Объединяем данные по колонкам "Вопрос", "Ответ" и "Контекст"
    df_merged = df[['Вопрос', 'Ответ', 'Контекст']]
    # Добавляем имя файла (модели) в отдельную колонку
    df_merged['Модель'] = file
    # Добавляем данные к общей таблице
    df_all = pd.concat([df_all, df_merged], ignore_index=True)

# Сохраняем общую таблицу в CSV и XLSX
df_all.to_csv(os.path.join(project_path, 'score_result.csv'), index=False)
df_all.to_excel(os.path.join(project_path, 'score_result.xlsx'), index=False)